In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **GROUP 3 PROJECT PROPOSAL**

# Introduction


The dataset contains details of red and white winho verde wine samples from the north of Portugal and quality is based on the physicochemical tests. 

Dataset citation: P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009

The question that we will be trying to answer using our dataset is predicting what the quality of a wine would be given its wine features (physicochemical tests such as pH level, alcohol level, etc). And we will be doing classification. 


# Preliminary exploratory data analysis


In [25]:
import pandas as pd
from sklearn.model_selection import (
    train_test_split,
)

In [26]:
data_red = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data_white = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")
data_red.head(3)
#data_red.sample(n=3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [27]:
#data_white.sample(n=3)
data_white.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [35]:
red_train, red_test = train_test_split(data_red, test_size=0.25, random_state=123)
white_train, white_test = train_test_split(data_white, test_size=0.25, random_state=123)
red_train.head()
X_red_train = red_train.drop('quality', axis=1)
y_red_train = red_train['quality']
X_white_train = white_train.drop('quality', axis=1)
y_white_train = white_train['quality']
X.head()
y.head()

502     7
585     6
258     5
1486    5
831     6
Name: quality, dtype: int64

- loaded data into notebook and read
- tidy data: each variable forms a volumn & each observation forms a row & each cell is a single measurement
- separate data into X_training, y
- create a plot using the training data

# Methods


# Expected outcomes and significance
